#### To begin, run the cell below. It contains all of the necessary set-up code.

In [ ]:
import mysql.connector
import sqlalchemy as db
from datetime import datetime
import pandas as pd
import numpy as np
from pandas.tseries.offsets import MonthEnd
from pandas.tseries.offsets import YearEnd
import openpyxl
from openpyxl.styles import Font, Alignment, Border, Side
from dateutil.relativedelta import relativedelta

#database path
f = open("db_path.txt", "r")
path = f.read()
f.close()

#create sqlalchemy engine for pd.read_sql
engine = db.create_engine(path)

####DEFINE COMPANY CLASS#####################################################
class company:
    
    def __init__(self, engine, company_id, FYE_month):
        
#########RETREIVE DATA#######################################################

        #establish connection to db
        self.engine = engine
        self.company_id = company_id
        
        ###query leases based on company_id
        self.lease_info = pd.read_sql_query ('select * from leases WHERE company_id = %(identifier)s', engine.connect(), params = {'identifier':company_id})
        ###convert datetime fields
        self.lease_info['commencement_date'] = pd.to_datetime(self.lease_info['commencement_date'])
        self.lease_info['termination_date'] = pd.to_datetime(self.lease_info['termination_date'])
        
        ###query payments based on company_id
        self.payments = pd.read_sql_query ('select * from payments WHERE company_id = %(identifier)s', engine.connect(), params = {'identifier':company_id})
        ###convert datetime field
        self.payments['payment_date'] = pd.to_datetime(self.payments['payment_date'])

########LEASE CLASSIFICATION & START OF AMORTIZATION SCHEDULES################

        #start amortization df
        self.amort = pd.DataFrame()
        
        #start self.amort by getting all relevant dates. These are payments dates & reporting dates (month ends)
        for lease_id in self.lease_info['lease_id']:

            #get payments data by individual lease
            pmts = pd.DataFrame(self.payments.loc[self.payments['lease_id']==lease_id])

            #get start/end dates for the lease
            s = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'commencement_date'].item()   
            e = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'termination_date'].item() 
            e_m = pd.to_datetime(e) + MonthEnd(0)
            
            s_y = s - pd.offsets.YearEnd(month = FYE_month)

            if FYE_month != 12:
                e_y = e + pd.offsets.YearEnd(month = FYE_month)
            else: 
                e_y = e - pd.offsets.YearEnd()
            
            #label existing data as payment data
            pmts['date_type'] = 'payment'
            
            #get the monthly reporting dates and label them
            rpt_m = pd.DataFrame(pd.date_range(start = s, end = e_m,  freq = 'M'), columns = ['payment_date'])
            rpt_m['date_type'] = 'monthly_reporting'
        
            #get the annual reporting dates and label them
            rpt_y = pd.DataFrame(pd.date_range(start = s_y, end = e_y,  freq = 'Y') + pd.offsets.YearEnd(month=FYE_month), columns = ['payment_date'])
            rpt_y['date_type'] = 'annual_reporting'

            #combine the three dataframes, sort, and handle na values
            temp = pd.merge(pmts, rpt_m, how = 'outer', on = 'payment_date')
            temp = pd.merge(temp, rpt_y, how = 'outer', on = 'payment_date')
            temp = temp.sort_values(by = 'payment_date')
            temp['payment_amount'] = temp['payment_amount'].fillna(0)
            temp[['company_id', 'lease_id']] = temp[['company_id', 'lease_id']].fillna(method = 'ffill')
            
            #add cumulative payment information
            temp['Cumulative Lease Payments'] = temp['payment_amount'].cumsum()
            
            #add to amortization df and delete temporary variables
            self.amort = pd.concat([self.amort, temp])
            del temp
            del rpt_m
            del rpt_y
            del pmts
        
        #clean up self.amort columns
        self.amort.rename(columns = {'date_type_x': 'payment', 'date_type_y': 'monthly_rpt', 'date_type': 'annual_rpt'}, inplace = True)
        self.amort.drop(columns = ['company_id'], inplace = True)

        #add information for the present value calculation into self.amort
        self.amort = pd.merge(self.amort, 
                              self.lease_info[['lease_id','discount_rate','commencement_date', 'termination_date']],
                              how = 'inner',
                              on = 'lease_id')
        
        #use the above information to calculate days from lease inception for each date in self.amort
        self.amort['Days From Inception'] = self.amort['payment_date'] - self.amort['commencement_date']
        
        self.amort['Days From Inception'] = np.where((self.amort['payment_date'] > self.amort['termination_date']),
                                                     self.amort['termination_date'] - self.amort['commencement_date'],
                                                     self.amort['Days From Inception'])
        
        self.amort['Days From Inception'] = self.amort['Days From Inception'].dt.days.astype('int16')
        
        #clean-up of termination_date column
        self.amort.drop(columns = ['termination_date'], inplace = True)
        
        #add present value of each individual payment to self.amort
        self.amort['Present Value'] = round(self.amort['payment_amount']/(1 + self.amort['discount_rate'])**(self.amort['Days From Inception']/365),2)

        #get sum of pvs for each lease
        pvs = self.amort.groupby(by = ['lease_id'])['Present Value'].sum()
        
        #store this sum of pvs for each lease in self.lease_info
        self.lease_info = pd.merge(self.lease_info, 
                                   pvs,
                                   how = 'inner',
                                   on = 'lease_id')
        #clear pv of payments out of self.amort
        self.amort.drop(columns = ['Present Value'], inplace = True)
        
        #clear the temporary varaible
        del pvs
        
        #conduct the fair value test
        self.lease_info['fv_test'] = 0
        self.lease_info['fv_test'] = np.where((self.lease_info['fair_value_of_asset']
                                               *self.lease_info['fair_value_policy']
                                               < self.lease_info['Present Value']),
                                              1, self.lease_info['fv_test'])
        
        #conduct the useful life test
        self.lease_info['useful_life_test'] = 0
        self.lease_info['useful_life_test'] = np.where((self.lease_info['useful_life_of_asset']
                                                        *self.lease_info['economic_life_policy']
                                                        < round((self.lease_info['termination_date']
                                                                 -self.lease_info['commencement_date'])
                                                                / np.timedelta64(1, 'M'), 0)),
                                                            1, self.lease_info['useful_life_test'])
        
        
        #create classification columns, default to operating classification
        self.lease_info['Secondary Classification'] = 'Operating'
        
        #if finance lease criteria apply, change to finance lease. Separate into finance lease 1 & 2 
        #based on ROU asset amortization method.
        
        for criteria in ['specialized_asset','fv_test','useful_life_test']:
            
            self.lease_info['Secondary Classification'] = np.where((self.lease_info[criteria]==1),
                                                         'Finance 1',
                                                         self.lease_info['Secondary Classification'])   

        for criteria in ['ownership_transfer','purchase_option']:
            
            self.lease_info['Secondary Classification'] = np.where((self.lease_info[criteria]==1),
                                                         'Finance 2',
                                                         self.lease_info['Secondary Classification'])
            
        #create a simple classification column
        self.lease_info['Classification'] = self.lease_info['Secondary Classification'].apply(lambda x: 'Operating' if x == 'Operating' else 'Finance')
    
    
    def get_interest(self, all_dates, starting_balance):
    
        temp_1 = all_dates.copy()

        #GET PERIOD BETWEEN DATES
        #get period between payments (in terms of 365 day year)
        temp_1['Days Between Dates'] = temp_1['Days From Inception'] - temp_1['Days From Inception'].shift()
        
        temp_1['Days Between Dates'] = np.where((temp_1['Days Between Dates'].isnull()),
                                            temp_1['Days From Inception'],
                                            temp_1['Days Between Dates'])
        
        temp_1['Period Between Dates'] = temp_1['Days Between Dates'] / 365

        #clean negatives (which occur at transitions between lease ids) & nans (which occur due to .shift() being used)
        temp_1['Period Between Dates'].fillna(0, inplace = True)

        #get interest expense factor
        temp_1['Interest Expense Factor'] = ((1+temp_1['discount_rate'])**temp_1['Period Between Dates'])-1

        #GET INTEREST EXPENSE
        #set up list to get interest expense
        interest_expense = []

        #define interest and liability related to each payment
        for payment, factor in zip(temp_1['payment_amount'], temp_1['Interest Expense Factor']):

            #calc interest and new starting balance (balance right after this payment)
            interest_component = round(starting_balance * factor, 2)
            starting_balance = round(starting_balance + interest_component - payment, 2)

            #check the liability balance. If the balance is below one, correct to 0.
            if starting_balance < 1:
                interest_component -= starting_balance
                starting_balance = 0

            #record the interest component
            interest_expense.append(interest_component)

        #record liability and interest amounts in temp_1
        temp_1['Interest'] = interest_expense
        temp_1['Cumulative Interest'] = temp_1['Interest'].cumsum()

        #drop interest expense factor (no longer needed)
        temp_1.drop(columns = ['Interest Expense Factor', 'Days Between Dates', 'Period Between Dates'], inplace = True)

        del interest_expense
        del interest_component

        return temp_1
    
    def amortize(self, reporting_dates_with_interest, starting_balance, classification, useful_life):
    
        temp_2 = reporting_dates_with_interest.copy()

        #GET PERIOD BETWEEN DATES
        #get period between payments (in terms of 365 day year)
        temp_2['Days Between Dates'] = temp_2['Days From Inception'] - temp_2['Days From Inception'].shift()

        temp_2['Days Between Dates'] = np.where((temp_2['Days Between Dates'].isnull()),
                                                temp_2['Days From Inception'],
                                                temp_2['Days Between Dates'])

        temp_2['Period Between Dates'] = temp_2['Days Between Dates'] / 365

        #clean negatives (which occur at transitions between lease ids) & nans (which occur due to .shift() being used)
        temp_2['Period Between Dates'].fillna(0, inplace = True)

        #get correct payment amounts for period
        temp_2['Lease Payments'] = temp_2['Cumulative Lease Payments'] - temp_2['Cumulative Lease Payments'].shift()
        temp_2['Lease Payments'] = np.where((temp_2['Lease Payments'].isna()), temp_2['Cumulative Lease Payments'], temp_2['Lease Payments']) 

        #get correct interest amounts for period
        temp_2['Period Interest'] = temp_2['Cumulative Interest'] - temp_2['Cumulative Interest'].shift()
        temp_2['Period Interest'] = np.where((temp_2['Period Interest'].isna()), temp_2['Cumulative Interest'], temp_2['Period Interest'])    

        #clean-up
        temp_2.drop(columns = ['Days Between Dates', 'Cumulative Lease Payments',
                               'Days From Inception', 'payment_id', 'payment_amount', 'payment', 'monthly_rpt',
                               'annual_rpt', 'discount_rate', 'commencement_date', 'Interest', 'Cumulative Interest'], inplace = True)

        temp_2.rename(columns = {'payment_date': 'Date', 'Period Interest':'Interest'}, inplace = True)


        #Get the liability balances
        lease_liability = []

        liab = starting_balance

        for payment, interest in zip(temp_2['Lease Payments'], temp_2['Interest']):

            liab = liab + interest - payment
            lease_liability.append(round(liab, 2))

        temp_2['Lease Liability'] = lease_liability

        #Get the asset balances
        if classification == 'Operating':
            base = temp_2['Lease Payments'].sum()
            period = temp_2['Period Between Dates'].sum()
            temp_2['Lease Expense'] = round(temp_2['Period Between Dates'] / period * base, 2)
            temp_2['ROU Asset Amortization'] = temp_2['Lease Expense'] - temp_2['Interest']
        elif classification == 'Finance 1':
            period = temp_2['Period Between Dates'].sum()
            temp_2['Lease Expense'] = round(temp_2['Period Between Dates'] / period * starting_balance,2)
            temp_2['ROU Asset Amortization'] = temp_2['Lease Expense']
        elif classification == 'Finance 2':
            base = useful_life / 12
            temp_2['Lease Expense'] = round(temp_2['Period Between Dates'] / base * starting_balance,2)
            temp_2['ROU Asset Amortization'] = temp_2['Lease Expense']


        temp_2 ['Accumulated Amortization'] = temp_2['ROU Asset Amortization'].cumsum()

        temp_2['Accumulated Amortization'] = np.where((starting_balance - temp_2['Accumulated Amortization'] < 1),
                                                      starting_balance,
                                                      temp_2['Accumulated Amortization'])

        temp_2['ROU Asset'] = starting_balance - temp_2['Accumulated Amortization']
        
        temp_2.drop(columns = ['Period Between Dates'], inplace = True)

        return temp_2
    
    def lease_schedule(self, lease_id, report_type = 'A', to_excel = False, save_path = None):
        
        starting_balance = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'Present Value'].item()
        classification = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'Secondary Classification'].item()
        useful_life = reporting_company.lease_info.loc[reporting_company.lease_info['lease_id']==lease_id, 'useful_life_of_asset'].item()
           
        a = self.amort.loc[self.amort['lease_id']==lease_id]
        b = self.get_interest(a, starting_balance)
        
        if report_type == 'A':
            c = b.loc[b['payment']=='payment']
        elif report_type == 'M':
            c = b.loc[b['monthly_rpt']=='monthly_reporting']
        elif report_type == 'Y':
            c = b.loc[b['annual_rpt']=='annual_reporting']
        else:
            print('Invalid Report Type')
            return
        
        
        df = self.amortize(c, starting_balance, classification, useful_life)
        df = df.reset_index(drop = True)
        
        if to_excel:
            assert len(save_path) > 0, 'You must provide a save path'
            
            df.drop(columns = 'lease_id', inplace = True)
            name = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'lease_name'].item()
            
            file_path = f"{save_path}/{name}.xlsx"
            
            with pd.ExcelWriter(file_path) as writer:  
                df.to_excel(writer, sheet_name=name)
                
            workbook = openpyxl.load_workbook(file_path)
            ws = workbook.active
            
            bottom = len(df) + 1
            
            for row in ws[f"B2:B{bottom}"]:
                for cell in row:
                    cell.number_format = 'yyyy-mm-dd'
            
            for row in ws[f"C2:I{bottom}"]:
                for cell in row:
                    cell.number_format = '#,##0'
            
            workbook.save(file_path)
                
            print(f"A spreadsheet titled '{name}.xlsx' has been sent to the specified file path.")
            return 
            
        return df
    
    def lease_reporting(self, lease_id):
    
        temp = self.lease_schedule(lease_id, report_type = 'Y')

        #fetch and check lease_id
        lease_id = temp['lease_id'].unique()
        assert len(lease_id) == 1
        lease_id = lease_id[0]

        #fetch classification
        classification = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'Classification'].item()

        #fetch present value
        amount = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'Present Value'].item()

        #fetch initial up-front payments
        init_payments = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'upfront_payments'].item()

        #Get beginning balance columns
        temp[f"{classification}: Beginning Lease Liability"] = temp['Lease Liability'].shift()
        temp[f"{classification}: Beginning ROU Asset"] = temp['ROU Asset'].shift()
        temp.fillna(0, inplace = True)

        #column set-up
        for c in ['Operating', 'Finance']:
            temp[f"{c}: Non Cash Lease Liability Obtained"] = 0
            temp[f"{c}: Non Cash ROU Asset Obtained"] = 0
            temp[f"{c}: Investment in ROU Asset (Cash)"] = 0

        #Get Lease Liability/ROU Asset Obtained
        temp[f"{classification}: Investment in ROU Asset (Cash)"] = np.where((temp[f"{classification}: Beginning Lease Liability"]==0),
                                                                      init_payments,
                                                                      temp[f"{classification}: Investment in ROU Asset (Cash)"])

        temp[f"{classification}: Non Cash ROU Asset Obtained"] = np.where((temp[f"{classification}: Beginning Lease Liability"]==0),
                                                                      amount-init_payments,
                                                                      temp[f"{classification}: Non Cash ROU Asset Obtained"])

        temp[f"{classification}: Non Cash Lease Liability Obtained"] = np.where((temp[f"{classification}: Beginning Lease Liability"]==0),
                                                                                amount-init_payments,
                                                                                temp[f"{classification}: Non Cash Lease Liability Obtained"])

        #Get cash flow information (Liability)
        temp[f"{classification}: Lease Repayments"] = (temp['Lease Payments'] - temp['Interest']-temp[f"{classification}: Investment in ROU Asset (Cash)"])*-1
        temp[f"{classification}: Ending Lease Liability"] = temp['Lease Liability']

        #Get cash flow information (Assets)
        temp[f"{classification}: ROU Asset Amortization"] = -(temp['ROU Asset Amortization'])
        temp[f"{classification}: Ending ROU Asset"] = temp['ROU Asset']

        #Get balance sheet information (Liability)
        temp[f"{classification}: Lease Liability, Current Portion"] = temp['Lease Liability'] - temp['Lease Liability'].shift(-1)
        temp[f"{classification}: Lease Liability"] = temp['Lease Liability'] - temp[f"{classification}: Lease Liability, Current Portion"]

        temp.fillna(0, inplace = True)

        #Get Income Statement Information
        temp[f"{classification}: Lease Expense"] = temp['Lease Expense']
        temp[f"{classification}: Interest Expense"] = temp['Interest']
        temp[f"{classification}: Amortization Expense"] = temp[f"{classification}: ROU Asset Amortization"]*-1

        #Get Future Imputed Interest for Future Minimum Payments Table
        temp['temp_interest'] = temp['Interest'].sum()
        temp['temp_interest2'] = temp['Interest'].cumsum()
        temp[f"{classification}: Future Imputed Interest"] = temp['temp_interest'] - temp['temp_interest2']
        temp[f"{classification}: Lease Payments by Year"] = temp['Lease Payments']
        temp.drop(columns = ['temp_interest', 'temp_interest2'], inplace = True)

        #get discount_rate column
        end = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'termination_date'].item()
        rate = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'discount_rate'].item()
        temp[f"{classification}: Discount Rate"] = rate
        temp[f"{classification}: Discount Rate"] = np.where((temp['Date'] > end), 0, temp[f"{classification}: Discount Rate"])
        
        #get term remaining
        temp['Term Remaining Y'] = temp['Date'].apply(lambda x: relativedelta(end, x).years)
        temp['Term Remaining M'] = temp['Date'].apply(lambda x: relativedelta(end, x).months)
        temp['Term Remaining M'] = round(temp['Term Remaining M'] / 12, 2)
        temp['Term Remaining M'] = temp['Term Remaining M'].apply(lambda x: 0 if x < 0 else x)
        temp[f"{classification}: Term Remaining"] = temp['Term Remaining Y'] + temp['Term Remaining M']
        temp.drop(columns = ['Term Remaining Y', 'Term Remaining M'], inplace = True)
        
        #Disclosure Tables
        bs_a = temp[['Date', f"{classification}: Ending ROU Asset"]]

        bs_l = temp[['Date', f"{classification}: Lease Liability, Current Portion", f"{classification}: Lease Liability"]]

        if classification == 'Operating':
            ops = temp[['Date',f"{classification}: Lease Expense"]]
        elif classification == 'Finance':
            ops = temp[['Date', f"{classification}: Amortization Expense", f"{classification}: Interest Expense"]]

        socf_a = temp[['Date', f"{classification}: Beginning ROU Asset", f"{classification}: Investment in ROU Asset (Cash)",
                      f"{classification}: Non Cash ROU Asset Obtained", f"{classification}: ROU Asset Amortization",
                      f"{classification}: Ending ROU Asset"]]

        socf_l = temp[['Date', f"{classification}: Beginning Lease Liability", f"{classification}: Non Cash Lease Liability Obtained",
                      f"{classification}: Lease Repayments", f"{classification}: Ending Lease Liability"]]

        fmp = temp[['Date', f"{classification}: Lease Payments by Year", f"{classification}: Future Imputed Interest"]]
        
        dr = temp[['Date', f"{classification}: Discount Rate"]]
        
        tr = temp[['Date', f"{classification}: Term Remaining"]]
        
        return (bs_a, bs_l, ops, socf_a, socf_l, fmp, dr, tr)

    def rename_disclosure_table(self, lease_id, df):
    
        name = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'lease_name'].item()
        temp = df.copy()

        for col in list(temp.columns)[1:]:
            temp.rename(columns = {col: f"{col}: {name}"}, inplace = True)

        return temp

    def get_col_names (self, df):

        col_vals = []

        for i in list(df.columns)[1:]:
            val = i.split(":")[1].strip()
            col_vals.append(val)

        return col_vals

    def format_df(self, b_col, b, classification):

        b_list = []

        for simple_col in b_col:

            new_df = pd.DataFrame()
            new_df['Date'] = b[['Date']]

            for detailed_col in list(b.columns)[1:]:
                if detailed_col.split(':')[1].strip() == simple_col:
                    new_df[detailed_col] = b[detailed_col]

            new_df[f"{classification}: {simple_col}: Total"] = new_df.sum(numeric_only = True, axis = 1)

            b_list.append(new_df)

        final_df = pd.DataFrame()
        final_df['Date'] = b[['Date']]

        for df in b_list:
            final_df = pd.merge(final_df, df, how = 'outer', on = 'Date')

        return final_df
    
    def weighted_average(self, df):
        
        temp = df.copy()
        temp = temp.T
        weighted_averages = []
        
        for col in list(temp.columns):
            temp['weight'] = temp[col].sum()
            temp['weight'] = temp[col] / temp['weight']
            temp['weight'] = temp['weight'] * temp[col]
            weighted_averages.append(round(temp['weight'].sum(), 4))

        return weighted_averages
    
    def company_disclosures(self, classification):
    
        leases = list(self.lease_info[self.lease_info['Classification']==classification]['lease_id'])

        if len(leases) > 0:

            reports = self.lease_reporting(leases[0])
            dates = reports[0]['Date']

            a = pd.DataFrame(dates)

            b = pd.DataFrame(dates)
            b_col = self.get_col_names(reports[1])

            c = pd.DataFrame(dates)
            c_col = self.get_col_names(reports[2])

            d = pd.DataFrame(dates)
            d_col = self.get_col_names(reports[3])

            e = pd.DataFrame(dates)
            e_col = self.get_col_names(reports[4])

            f = pd.DataFrame(dates)
            f_col = self.get_col_names(reports[5])
            
            g = pd.DataFrame(dates)
            g_col = self.get_col_names(reports[6])
            
            h = pd.DataFrame(dates)
            h_col = self.get_col_names(reports[7])
            

            merge = lambda old, new, lease_id: pd.merge(old, self.rename_disclosure_table(lease_id, new), how = 'outer', on = 'Date')

            for lease_id in leases:

                tables = self.lease_reporting(lease_id)

                a = merge(a, tables[0], lease_id)
                b = merge(b, tables[1], lease_id)
                c = merge(c, tables[2], lease_id)
                d = merge(d, tables[3], lease_id)
                e = merge(e, tables[4], lease_id)
                f = merge(f, tables[5], lease_id)
                g = merge(g, tables[6], lease_id)
                h = merge(h, tables[7], lease_id)
        else: 
            print('The Company does not have any leases with that classification')
            return 0

        a[f"{classification}: Ending ROU Asset: Total"] = a.sum(numeric_only = True, axis = 1)
        b = self.format_df(b_col, b, classification)
        c = self.format_df(c_col, c, classification)
        d = self.format_df(d_col, d, classification)
        e = self.format_df(e_col, e, classification)
        f = self.format_df(f_col, f, classification)
        
        return_tuple = (a, b, c, d, e, f, g, h)

        for df in return_tuple:
            df.sort_values(by='Date', inplace = True)
            df.set_index('Date', inplace = True)
            df.fillna(0, inplace = True)
        
        g[f"{classification}: Discount Rate: Weighted Average: Total"] = self.weighted_average(g)
        h[f"{classification}: Term Remaining: Weighted Average: Total"] = self.weighted_average(h)

        return (a, b, c, d, e, f, g, h)

    def audit_reports (self, save_path, report_type = 'A'):
        
        file_path = f"{save_path}/audit_reports.xlsx"
        
        ##User Inputs
        user_inputs = self.lease_info[list(self.lease_info.columns)[2:15]]
        
        with pd.ExcelWriter(file_path) as writer:  
                user_inputs.to_excel(writer, sheet_name='User Inputs')        
        
        ##Lease Summary
        lease_summary = pd.DataFrame(self.lease_info[['lease_name', 'commencement_date',
                                         'termination_date', 'Classification']])
        lease_summary.rename(columns = {'lease_name':"Lease Name", 'commencement_date': 'Commencement Date',
                                        'termination_date':'Termination Date'}, inplace = True)
        
        with pd.ExcelWriter(file_path,
                            mode = 'a',
                            engine="openpyxl",
                            if_sheet_exists="overlay") as writer:
            
            lease_summary.to_excel(writer, sheet_name='Lease Summary')
        
        
        amortization_lengths = {}
        
        with pd.ExcelWriter(file_path,
                    mode = 'a',
                    engine="openpyxl",
                    if_sheet_exists="overlay") as writer:
            
            for lease_id in list(self.lease_info['lease_id']):
                name = self.lease_info.loc[self.lease_info['lease_id']==lease_id, 'lease_name'].item()
                df = self.lease_schedule(lease_id, report_type)
                amortization_lengths[name]=len(df)+1
                df.drop(columns = 'lease_id', inplace = True)
                df.to_excel(writer, sheet_name=f"{name}")
                
        #Disclosure Tables - Creating Sheets & Formatting
        for classification in ['Operating', 'Finance']:
            
            #get disclosure tables
            tables = self.company_disclosures(classification)
            
            #make sure there are leases of this type
            if tables == 0:
                continue
            
            #start variables to house information for formatting
            startrow = 2
            startrows = {}
            width = 0
            
            with pd.ExcelWriter(file_path,
                        mode = 'a',
                        engine="openpyxl",
                        if_sheet_exists="overlay") as writer:

                for df in tables:
                    
                    #record rows for formatting
                    startrows[startrow+1] = [0]
                    
                    #get width for formatting
                    width = max(len(df), width)
                    
                    #transpose DF
                    df = df.T
                    
                    #get 'total' rows for formatting
                    rows = list(df.index)
                    for loc, row in enumerate(rows):
                        split_row = row.split(':')
                        split_row = [i.strip() for i in split_row]
                        if split_row[-1] == 'Total':
                            startrows[startrow+1].append(loc + 1)

                    #write table into excel
                    df.to_excel(writer, sheet_name=f"{classification}_Lease_Disclosures", startrow = startrow)
                    
                    #set start row for next table
                    startrow += len(df) + 3
            
            #Format Disclosure Tables
            #get table titles
            titles = ['Balance Sheet - Asset', 'Balace Sheet - Liability', 'Income Statement',
            'Cash Flows - Asset', 'Cash Flows - Liability', 'Future Minimum Payments',
            'Weighted Average Discount Rate', 'Weighted Average Remaining Lease Term']
            
            #get active worksheet
            workbook = openpyxl.load_workbook(file_path)
            workbook.active = workbook[f"{classification}_Lease_Disclosures"]
            ws = workbook.active
            
            #get rightmost boundary of the tables
            right = chr(ord('a') + width).upper()
            
            #remove all borders from active worksheet
            for row in ws.iter_rows():
                for cell in row:
                    cell.border = None
                    
            for start, title in zip(startrows.keys(), titles):

                thin = Side(border_style="thin", color="000000")

                top = start
                bottom = top + startrows[start][-1]

                #get the date format
                for row in ws[f"B{top}:{right}{top}"]:
                    for cell in row:
                        cell.alignment = Alignment(horizontal = 'center')
                        cell.number_format = 'yyyy-mm-dd'

                #get the number format
                if title[0] == 'W':
                    for row in ws[f"B{top+1}:{right}{bottom}"]:
                        for cell in row:
                            cell.number_format = '0.0000'
                else:
                    for row in ws[f"B{top+1}:{right}{bottom}"]:
                        for cell in row:
                            cell.number_format = '#,##0'

                #column A remove bold and align right
                for row in ws[f"A{top}:A{bottom}"]:
                    for cell in row:
                        cell.font = Font(bold = False)
                        cell.alignment = Alignment(horizontal = 'right')

                #give the table a title
                ws[f"A{start}"] = title

                #format total rows
                for loc in startrows[start]:

                    #set target row
                    target = start + loc

                    #set the row bold & add bottom/top borders
                    for row in ws[f"A{target}:{right}{target}"]:
                        for cell in row:
                            cell.font = Font(bold=True)
                            cell.border = Border(top = thin, bottom = thin)

                    #align left the total labels
                    ws[f"A{target}"].alignment = Alignment(horizontal = 'left')
            
            #Format amortization schedules
            
            names = list(self.lease_info['lease_name'])
            
            for name in names:
                
                workbook.active = workbook[name]
                ws = workbook.active
                
                bottom = amortization_lengths[name]
            
                for row in ws[f"B2:B{bottom}"]:
                    for cell in row:
                        cell.number_format = 'yyyy-mm-dd'
            
                for row in ws[f"C2:I{bottom}"]:
                    for cell in row:
                        cell.number_format = '#,##0'
                        
            workbook.active = workbook['User Inputs']
            ws = workbook.active
            bottom = len(user_inputs)+1
            
            for row in ws[f"C2:D{bottom}"]:
                for cell in row:
                    cell.number_format = 'yyyy-mm-dd'
            for row in ws[f"E2:N{bottom}"]:
                for cell in row:
                    cell.number_format = '#,##0'
            for row in ws[f"G2:G{bottom}"]:
                for cell in row:
                    cell.number_format = '0.0000'
            for row in ws[f"J2:K{bottom}"]:
                for cell in row:
                    cell.number_format = '0.00'
            
            #Format Lease Summary
            workbook.active = workbook['Lease Summary']
            ws = workbook.active
            bottom = len(lease_summary) + 1
            
            for row in ws[f"C2:D{bottom}"]:
                for cell in row:
                    cell.number_format = 'yyyy-mm-dd'
            
            #save and close the workbook
            workbook.save(file_path)
            workbook.close()
        return        

## Step 1 - Select a Company

Select a company to pull reports.

1. Run the cell below to view existing companies.

In [ ]:
companies = pd.read_sql_query ('select * from companies', engine.connect())
pd.set_option('display.max_rows', None)
display(companies)

2. Enter the desired company's name as a string. 

3. Additionally, enter the last month of the fiscal year as an integer (i.e., 12 for December).

In [ ]:
company_name = 'Zenith Digital Solutions'
fye_month = 12

In [ ]:
try:
    company_id = companies.loc[companies['company_name']==company_name, 'company_id'].item()
except:
    print('The company name selected does not appear in the table above. Please enter a valid company name.')

assert fye_month > 0 and fye_month <=12, 'You have not entered a valid number for the last month of the fiscal year'    

print(f"You have successfully selected {company_name}. The companies leases are listed below.")

reporting_company = company(engine, company_id, fye_month)
display(reporting_company.lease_info[['lease_id', 'lease_name']])

## Step 2: Pull Reports

Enter the path to the location where reports should be exported.

In [ ]:
save_path = '/Users/painelewis/Desktop'

### 1. Lease Reports

Follow instructions below to pull reports for an individual lease.

> Provide the following inputs:
>> <b>lease_id</b>: Enter the desired lease id (from the table above) as an integer.
<br/>
<br/>
>> <b>report_type</b>: Enter one of the following strings:
>>> 'A': For a traditional amortization schedule
<br/>
>>> 'M': For a monthly financial reporting summary
<br/>
>>> 'Y': For a yearly financial reporting summary


>> <b>to_excel</b>: Enter a boolean (True or False) to indicate whether the report should be to be exported to excel. Entering False will display the report below the cell.

After entering the correct input, run the two cells below.

In [ ]:
lease_id = 19
report_type = 'A'
to_excel = False

In [ ]:
reporting_company.lease_schedule(lease_id, report_type, to_excel, save_path)

### 2. Company Report

The cell below creates an excel spreadsheet that has the following:
1. All user inputs related to each lease on one tab
1. A summary of the Company's leases
1. A tab for each lease summarizing lease activity (either as a standard amortization schedule, or as a monthly/annual reporting summary based on the specification provided). 
1. Tabs summarizing all lease activity for the company segregated by operating and finance leases. 

To run the report, specify what information you would like at the individual lease level in the cell below. Then, run the following two cells. The file name will be 'audit_reports' and will appear in the specified file path. 

> <b> report type options:</b>
> - 'A' for standard amortization schedule
> - 'M' for monthly reporting summary
> - 'Y' for annual reporting summary

In [ ]:
report_type = 'A'

In [ ]:
reporting_company.audit_reports(save_path, report_type)